In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.2.10         
astor                         0.8.1          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.2.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.8.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bleach                        3.2.1          
blis                          0.4.

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics

df_train = pd.read_csv('drive/My Drive/MDE/NOTAS_E_NUMERO_DE_ACERTOS/NOTAS_E_NUMERO_DE_ACERTOS_2018.csv', sep="," , encoding="UTF8", skiprows=[i for i in range(41570,59385)])
print(df_train.shape)
df_test = pd.read_csv('drive/My Drive/MDE/NOTAS_E_NUMERO_DE_ACERTOS/NOTAS_E_NUMERO_DE_ACERTOS_2018.csv', sep="," , encoding="UTF8", skiprows=[i for i in range(1,41569)] )
print(df_test.shape)
df_test = df_test.drop(df_test.columns[[1,2,3,4]], axis=1)
print(df_test.shape)

(41570, 9)
(17817, 9)
(17817, 5)


**Quais colunas temos na base de test para trabalhar ?**

In [ ]:
df_test.columns

Index(['Unnamed: 0', 'ACERTOS_CN', 'ACERTOS_CH', 'ACERTOS_LC', 'ACERTOS_MT'], dtype='object')

**Vamos verificar quais features tem uma maior correlação com o nosso target (NU_NOTA_MT)**

In [ ]:
df_train.corr()

,Unnamed: 0,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,ACERTOS_CN,ACERTOS_CH,ACERTOS_LC,ACERTOS_MT
Unnamed: 0,1.000000,-0.030293,-0.043323,-0.042350,-0.007119,-0.020440,-0.037459,-0.038156,-0.005255
NU_NOTA_CN,-0.030293,1.000000,0.574202,0.603700,0.588234,0.896986,0.651859,0.628538,0.558796
NU_NOTA_CH,-0.043323,0.574202,1.000000,0.696338,0.546646,0.511705,0.889366,0.687950,0.491298
NU_NOTA_LC,-0.042350,0.603700,0.696338,1.000000,0.578378,0.534531,0.709415,0.950537,0.518359
NU_NOTA_MT,-0.007119,0.588234,0.546646,0.578378,1.000000,0.559916,0.624515,0.597624,0.893087
ACERTOS_CN,-0.020440,0.896986,0.511705,0.534531,0.559916,1.000000,0.614624,0.569459,0.560265
ACERTOS_CH,-0.037459,0.651859,0.889366,0.709415,0.624515,0.614624,1.000000,0.738290,0.588883
ACERTOS_LC,-0.038156,0.628538,0.687950,0.950537,0.597624,0.569459,0.738290,1.000000,0.548646
ACERTOS_MT,-0.005255,0.558796,0.491298,0.518359,0.893087,0.560265,0.588883,0.548646,1.000000


As features com maiores correlações em relação ao nosso Target (NU_NOTA_MT) serão as melhores a serem usadas, porém somente a seleção delas não vai garantir que sua análise seja satisfatória. De qualquer forma, vamos se atentar a selecionar as melhores nesse momento.


**Verificar como está as distribuições**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

features= [
      'ACERTOS_CN',
      'ACERTOS_CH',
      'ACERTOS_LC',
      'ACERTOS_MT']
      
features_corr= [
      'NU_NOTA_CN',
      'ACERTOS_CN',
      'NU_NOTA_CH',
      'ACERTOS_CH',
      'NU_NOTA_LC',
      'ACERTOS_LC',
      'NU_NOTA_MT',
      'ACERTOS_MT',]



**Verificando os valores nulos:**

In [ ]:
df_train[features].isnull().sum()

ACERTOS_CN    0
ACERTOS_CH    0
ACERTOS_LC    0
ACERTOS_MT    0
dtype: int64

**São muitos valores nulos!**

**Criando um heatmap das Features incluindo NU_NOTA_MT para vermos as nossas correlações:**


In [ ]:
corr = df_train[features_corr].corr()
ax = plt.subplots(figsize=(11, 8))
sns.heatmap(corr,  annot=True, annot_kws={"size": 10})


#Criando o modelo de regressão 

**Seguindo**:

    -Atribuir y_train ao meu target
    -Setar x test e x train com as features selecionadas
    -Normalizar os dados

In [ ]:
#y_train = df_train['NU_NOTA_MT']
y_train = df_train[['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT']]
x_train = df_train[features]
x_test = df_test[features]
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()  
print("\nantes")
print(x_train)
print(x_test)
x_train = sc.fit_transform(x_train)  
print("\ndepois")
print(x_train[0])
# x_test = sc.transform(x_test)
# print(x_test[0])


antes
       ACERTOS_CN  ACERTOS_CH  ACERTOS_LC  ACERTOS_MT
0              13          18          16           6
1               9          13          13          13
2              10           7          12           7
3              15          26          20          12
4              12          20          24          10
...           ...         ...         ...         ...
41565           9          23          12          17
41566           6          14          13          10
41567           8           8          10          10
41568          15          25          26          12
41569           6           6           5           5

[41570 rows x 4 columns]
       ACERTOS_CN  ACERTOS_CH  ACERTOS_LC  ACERTOS_MT
0              15          25          26          12
1              15          16          22          16
2              10          15          18           8
3              11          10           6           6
4              13          20          14        

In [ ]:
from sklearn.externals.joblib import dump, load

dir = "drive/My Drive/MDE/MODELOS_DE_PREDICAO_2016_A_2019/"

dump(sc, dir+'scaler.bin', compress=True)

#dump(sc, open(dir+'scaler.pkl', 'wb'))

['drive/My Drive/MDE/MODELOS_DE_PREDICAO_2016_A_2019/scaler.bin']

O modelo que usei para fazer a regressão, no caso foi o **RandomForestRegressor**. Mais informações sobre o modelo e os parâmetros você pode ver na  [documentação.](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)



In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor( 
           criterion='mae', 
           max_depth=8,
           max_leaf_nodes=None,
           min_impurity_split=None,
           min_samples_leaf=1,
           min_samples_split=2,
           min_weight_fraction_leaf=0.0,
           n_estimators= 100,
           n_jobs=-1,
           random_state=0,
           verbose=0,
           warm_start=False
)    



**Treinando o nosso modelo através do fit.**

In [ ]:
import joblib
regressor.fit(x_train, y_train) 

filename = 'finalized_model_2018.sav'
joblib.dump(regressor, filename)

['finalized_model_2018.sav']

**Realizando a predição das notas da nossa base test:**



In [ ]:
y_pred_test = regressor.predict(x_test)
y_pred_train = regressor.predict(x_train)

O **Codenation** não disponibiliza ou comenta qual o meio que o sistema avalia as notas. Dessa forma recomendo você criar o seu próprio método de avaliação e comparar com o score de submissão.

Eu usei o MAE, MSE E RSME do metrics para analisar, você pode achar a documentação aqui.

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_train, y_pred_train).round(8)  )
print('MSE:', metrics.mean_squared_error(y_train, y_pred_train).round(8) )  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)).round(8))
print(np.around(y_pred_test,2))
print(len(y_pred_test))
print(y_pred_test.shape)
df_test[['NU_NOTA_CN_PREDICTED','NU_NOTA_CH_PREDICTED','NU_NOTA_LC_PREDICTED','NU_NOTA_MT_PREDICTED']] = np.around(y_pred_test,2)
print(df_test.shape)
df_test.to_csv('NOTAS_CN_2019-PREDICTED.csv', index = False, header=True)

NameError: ignored

In [ ]:
import joblib

filename = 'finalized_model_2019.sav'
joblib.dump(regressor, filename)

['finalized_model_2019.sav']

In [ ]:
import joblib
loaded_model = joblib.load('drive/My Drive/MDE/MODELOS_DE_PREDICAO_2016_A_2019/finalized_model_2018.sav')
y_pred_test = loaded_model.predict(x_test)
y_pred_train = loaded_model.predict(x_train)

print(df_test)
print(x_test[0])
print(y_pred_test[0])
#[0.80237717 1.43889361 1.55658259 0.09492648]

       Unnamed: 0  ACERTOS_CN  ACERTOS_CH  ACERTOS_LC  ACERTOS_MT
0           41569          15          25          26          12
1           41570          15          16          22          16
2           41571          10          15          18           8
3           41572          11          10           6           6
4           41573          13          20          14          12
...           ...         ...         ...         ...         ...
17812       59381           9          12          10          10
17813       59382          10           8           6          15
17814       59383          10           6           9           7
17815       59384          11          27          23          11
17816       59385           6           6           5           5

[17817 rows x 5 columns]


KeyError: ignored

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_train, y_pred_train).round(8)  )
print('MSE:', metrics.mean_squared_error(y_train, y_pred_train).round(8) )  
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train)).round(8))

MAE: 24.83486201
MSE: 1141.21139769
RMSE: 33.78182052
